## ***מחברת קוד לעיבוד נתוני תוצאות מכון הנפט - סקר קרקע***

<bdi>

ברירת המחדל של הטבלה הינה הצגת חריגות מערך ה-VSL.

ניתן להציג חריגות מערך סף נוסף בהתאם למאפייני האתר.

בתא השני נבחר את התרכובות אותם נרצה להציג בטבלת התוצאות. לדוגמה: TPH, MTBE BTEX וכו'.

סימון האופציה **Add_exceptations** יציג בטבלה את כל תרכובות ה-VOC בהם יש חריגות מערכי הסף הנבחרים, VSL ו-TIER 1.

ב-**Add_specifics** נוסיף מספרי CAS של תרכובות נוספות אותן נרצה להציג על אף שלא אותרו בהן חריגות מערכי הסף.


על מנת לעשות זאת, יש לרשום במקום המיועד לכך לפי הפורמט הבא:

'cas number, cas number'

 לדוגמה: '106-93-4, 74-83-9'

 ניתן הוסיף כמה אנליזות שרוצים, 1,2,3 וכו'.

 אם תבוקש אנליזה שלא נמצאת בתוצאות מעבדה תקפוץ הודעה, יש ללחוץ אישור ולבדוק איזה אנליזה לא באמת קיימת בקובץ תוצאות שהעלתם

</bdi>

In [1]:
#@title נא לבחור פרמטרים מתאימים  { run: "auto" }

Tier1_type = "Tier 1 Residential A 0-6m" # @param ["Tier 1 Residential A 0-6m","Tier 1 Residential A >6m", "Tier 1 Residential B","Tier 1 industrial A 0-6m","Tier 1 industrial A >6m","Tier 1 industrial B"]

testsׁ_basic = "TPHּּּ + MTBE BTEX" # @param ["TPH","MTBE BTEX","TPHּּּ + MTBE BTEX"]

Add_exceptations_VOCs = True # @param {type:"boolean"}

Add_specifics_VOCs = None # @param {type:"raw"}

file_name = 'גל נתניה' # @param {type:"string"}

Font_size_hebrew = 9 # @param {type:"slider", min:6, max:24, step:1}

Font_name_hebrew = 'David' # @param {type:"string"}

Font_size_english = 8 # @param {type:"slider", min:6, max:24, step:1}

Font_name_english = 'Times New Roman' # @param {type:"string"}

!pip install openpyxl
from openpyxl.styles import Font
import pandas as pd
import re
from IPython.display import Javascript

if Add_specifics_VOCs == None:
  Add_specifics_VOCs = ''


In [2]:
#@title להעלות קובץ ערכי סף של קרקע בלחיצה על הכפתור למטה
from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        print(f"Unsupported file format: {filename}")
        continue
        # Round all numeric cells to 2 decimal points
    #df = df.applymap(lambda x: round(x, 2) if isinstance(x, (int, float)) else x)

    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    threshold_data = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")


Saving גרסה 7 דצמבר 2024 - soil_threshold.csv to גרסה 7 דצמבר 2024 - soil_threshold (1).csv


In [16]:
#@title להעלות קובץ תוצאות סקר קרקע של מכון הנפט בלחיצה על הכפתור למטה

from google.colab import files
uploaded = files.upload()

filenames = list(uploaded.keys())

# Initialize an empty list to store the dataframes
dataframes = []

# Loop through each file and read it based on its extension
for filename in filenames:
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
    elif filename.endswith('.xlsx'):
        df = pd.read_excel(filename)
    else:
        print(f"Unsupported file format: {filename}")
        continue
    dataframes.append(df)

# Concatenate all dataframes into one
if dataframes:
    df_res_all = pd.concat(dataframes, ignore_index=True)
else:
    print("No valid files uploaded.")

Saving 4949 VOC.xlsx to 4949 VOC.xlsx
Saving 5062 סונול איילון TPH+D+O+MTBE+BTEX.xlsx to 5062 סונול איילון TPH+D+O+MTBE+BTEX (1).xlsx
Saving 5102 סונול אור יהודה VOC.xlsx to 5102 סונול אור יהודה VOC (1).xlsx
Saving 5103 סונול המלאכה TPH+D+O+MBTE+BTEX.xlsx to 5103 סונול המלאכה TPH+D+O+MBTE+BTEX.xlsx
Saving 5207 סונול רמלה יינה TPH+D+O.xlsx to 5207 סונול רמלה יינה TPH+D+O (2).xlsx


In [17]:
#@title יש להריץ את התא ולהוריד את הטבלה המוכנה

def normalize_drill_name(item):
    if pd.isna(item):
        return item

    # Match patterns where the number in parentheses can be at the start or end
    match = re.match(r'\((\d+(\.\d+)?)\)\s*(.+)', item)
    if match:
        # Format as 'name (number)'
        return f"{match.group(3).strip()} ({match.group(1)})"
    else:
        # Check if it's already in correct format or in reverse
        match = re.match(r'(.+?)\s*\((\d+(\.\d+)?)\)', item)
        if match:
            return f"{match.group(1).strip()} ({match.group(2)})"
        else:
            # If no match, keep the item as is
            return item.strip()

results_data_TPH= pd.DataFrame()
results_data_MTBE= pd.DataFrame()
results_data_VOC= pd.DataFrame()
results_data_VOC_op = pd.DataFrame()

# Define the structure of the DataFrame
columns = ['name', 'date', 'Unnamed: 2', 'depth', ' PID','full name']
data = [
    ['units', None, None, 'm', 'ppm', None],
    ['Cas', None, None, None, None, None],
    ['VSL', None, None, None, None, None],
    ['TIER 1', None, None, None, None, None]
]

# Create the DataFrame
df_template = pd.DataFrame(data, columns=columns)

drills_list= []

if 'TPH' in testsׁ_basic:
    df_res_all_cleaned = df_res_all.reset_index(drop=True)

    collected_blocks = []  # To collect all TPH data blocks
    i = 0

    while i < len(df_res_all_cleaned):
        row = df_res_all_cleaned.iloc[i]

        # Look for 'בדיקה' in the row
        if 'בדיקה' in row.values:
            col_name_בדיקה = row[row == 'בדיקה'].index[0]
            col_index_בדיקה = df_res_all_cleaned.columns.get_loc(col_name_בדיקה)
            col_index_tph = col_index_בדיקה + 2

            # Confirm 'TPH' exists two columns to the right
            if col_index_tph < len(df_res_all_cleaned.columns):
                val = row.iloc[col_index_tph]
                if isinstance(val, str) and val.strip() == 'TPH':
                    selected_columns = [
                        df_res_all_cleaned.columns[col_index_בדיקה],       # 'בדיקה'
                        df_res_all_cleaned.columns[col_index_tph],         # 'TPH'
                        df_res_all_cleaned.columns[col_index_tph + 1],     # Next column
                        df_res_all_cleaned.columns[col_index_tph + 2],     # Next column
                    ]

                    start_row = i + 4
                    end_row = start_row

                    # Find the end of this block
                    for j in range(start_row, len(df_res_all_cleaned)):
                        if pd.isna(df_res_all_cleaned.iloc[j, col_index_בדיקה]):
                            break
                        end_row += 1

                    block = df_res_all_cleaned.loc[start_row:end_row - 1, selected_columns].copy()

                    # Rename and reorder columns
                    block.columns = ['Analyte', 'Total TPH', 'DRO', 'ORO']
                    block = block[['Analyte', 'DRO', 'ORO', 'Total TPH']]
                    block['Analyte'] = block['Analyte'].apply(normalize_drill_name)

                    collected_blocks.append(block)

                    # Move to end of current block
                    i = end_row
                    continue
        i += 1

    if collected_blocks:
        results_data_TPH = pd.concat(collected_blocks, ignore_index=True)
        drills_list = results_data_TPH["Analyte"].dropna().tolist()
    else:
        display(Javascript('alert("⚠️ TPH is not in the file results you uploaded ⚠️")'))

if 'MTBE' in testsׁ_basic:
    df_res_all_cleaned = df_res_all.reset_index(drop=True)
    collected_blocks = []
    i = 0

    while i < len(df_res_all_cleaned):
        row = df_res_all_cleaned.iloc[i]

        if 'בדיקה' in row.values:
            col_name_בדיקה = row[row == 'בדיקה'].index[0]
            col_index_בדיקה = df_res_all_cleaned.columns.get_loc(col_name_בדיקה)
            col_index_mtbe = col_index_בדיקה + 2

            if col_index_mtbe < len(df_res_all_cleaned.columns):
                val = row.iloc[col_index_mtbe]
                if isinstance(val, str) and val.strip() == 'MTBE':
                    selected_columns = [
                        df_res_all_cleaned.columns[col_index_בדיקה],       # 'בדיקה'
                        df_res_all_cleaned.columns[col_index_mtbe],        # MTBE
                        df_res_all_cleaned.columns[col_index_mtbe + 1],    # Benzene
                        df_res_all_cleaned.columns[col_index_mtbe + 2],    # Toluene
                        df_res_all_cleaned.columns[col_index_mtbe + 3],    # Ethylbenzene
                        df_res_all_cleaned.columns[col_index_mtbe + 4],    # Xylene
                    ]

                    start_row = i + 6
                    end_row = start_row

                    for j in range(start_row, len(df_res_all_cleaned)):
                        if pd.isna(df_res_all_cleaned.iloc[j, col_index_בדיקה]):
                            break
                        end_row += 1

                    block = df_res_all_cleaned.loc[
                        [start_row - 4] + list(range(start_row, end_row)),
                        selected_columns
                    ].copy()

                    block.columns = ['Analyte', 'MTBE', 'Benzene', 'Toluene', 'Ethylbenzene', 'Xylene']
                    collected_blocks.append(block)

                    i = end_row
                    continue
        i += 1

    if collected_blocks:
        results_data_MTBE = pd.concat(collected_blocks, ignore_index=True)

        if drills_list == []:
            results_data_MTBE['Analyte'] = results_data_MTBE['Analyte'].apply(normalize_drill_name)
            drills_list = results_data_MTBE["Analyte"].dropna().tolist()[1:]
        else:
            results_data_MTBE['Analyte'] = results_data_MTBE['Analyte'].apply(normalize_drill_name)
            mtbe_drills_list = results_data_MTBE["Analyte"].dropna().tolist()[1:]

            for mtbe_item in mtbe_drills_list:
                if not any(mtbe_item in drill_item for drill_item in drills_list):
                    drills_list.append(mtbe_item)

        results_data_MTBE = results_data_MTBE.reset_index(drop=True)

        def replace_ND(cell, column):
            if isinstance(cell, str) and "ND" in cell:
                return f"<{results_data_MTBE.iloc[0, column]}"
            return cell

        for col in results_data_MTBE.columns[1:]:
            results_data_MTBE[col] = results_data_MTBE[col].apply(
                lambda x: replace_ND(x, results_data_MTBE.columns.get_loc(col))
            )

        results_data_MTBE = results_data_MTBE.drop(index=0).reset_index(drop=True)

    else:
        if (df_res_all == '1634-04-4').any().any():
            df_res_all_cleaned = df_res_all.reset_index(drop=True)
            results_data_VOCs_dict = {}
            occurrence_count = 0

            for row_index, row in df_res_all_cleaned.iterrows():
                if 'ב ד י ק ה' in row.values:
                    col_name_בדיקה = row[row == 'ב ד י ק ה'].index[0]
                    col_index_בדיקה = df_res_all_cleaned.columns.get_loc(col_name_בדיקה)
                    col_index_גילוי = col_index_בדיקה + 4

                    if col_index_גילוי < len(df_res_all_cleaned.columns) and row.iloc[col_index_גילוי].strip() == 'גבול גילוי':
                        בדיקה_row = row_index
                        start_row = בדיקה_row + 3
                        end_row = start_row

                        for i in range(start_row, len(df_res_all_cleaned)):
                            value = df_res_all_cleaned.iloc[i, col_index_בדיקה + 1]
                            if pd.isna(value):
                                break
                            end_row += 1

                        start_col = col_index_גילוי + 2
                        end_col = start_col

                        for i in range(start_col, len(df_res_all_cleaned.columns)):
                            value = df_res_all_cleaned.iloc[בדיקה_row + 1, i]
                            if pd.isna(value):
                                break
                            end_col += 1

                        results_data_VOC = df_res_all_cleaned.iloc[
                            list(range(בדיקה_row + 1, end_row)),
                            list(range(col_index_בדיקה + 1, end_col))
                        ].reset_index(drop=True)

                        def replace_ND(cell, row_index):
                            if isinstance(cell, str) and "ND" in cell:
                                return f"<{results_data_VOC.iloc[row_index, 3]}"
                            return cell

                        for col in results_data_VOC.columns:
                            if results_data_VOC.columns.get_loc(col) != 3:
                                results_data_VOC[col] = results_data_VOC.apply(
                                    lambda row: replace_ND(row[col], row.name), axis=1
                                )

                        results_data_VOC_op = results_data_VOC.T.reset_index(drop=True)
                        results_data_VOC_op = results_data_VOC_op.drop(index=range(2, 5))
                        results_data_VOC_op = results_data_VOC_op.drop(columns=results_data_VOC_op.columns[1])
                        results_data_VOC_op = results_data_VOC_op.reset_index(drop=True)

                        if occurrence_count > 0:
                            results_data_VOC_op = results_data_VOC_op.drop(index=range(0, 2))

                        results_data_VOCs_dict[f"results_data_VOC_{occurrence_count}"] = results_data_VOC_op
                        occurrence_count += 1

            if results_data_VOCs_dict:
                results_data_VOC_op = pd.concat(results_data_VOCs_dict.values(), ignore_index=True)
                results_data_VOC_op = results_data_VOC_op.drop(index=1)

                cas_mapping = {
                    'MTBE': '1634-04-4',
                    'Benzene': '71-43-2',
                    'Toluene': '108-88-3',
                    'Ethylbenzene': '100-41-4',
                    'Xylene': '95-47-6'
                }

                cas_numbers = list(cas_mapping.values())
                columns_to_keep = [results_data_VOC_op.columns[0]]
                columns_to_keep += [
                    col for col in results_data_VOC_op.columns[1:]
                    if results_data_VOC_op.at[0, col] in cas_numbers
                ]
                results_data_VOC_op = results_data_VOC_op[columns_to_keep]

                for chemical, cas_number in cas_mapping.items():
                    results_data_VOC_op.replace(cas_number, chemical, inplace=True)

                results_data_VOC_op.columns = results_data_VOC_op.iloc[0]
                results_data_VOC_op = results_data_VOC_op[1:]
                results_data_VOC_op.rename(columns={results_data_VOC_op.columns[0]: 'Analyte'}, inplace=True)
                desired_order = ['Analyte', 'MTBE', 'Benzene', 'Toluene', 'Ethylbenzene', 'Xylene']
                results_data_VOC_op = results_data_VOC_op.reindex(columns=desired_order)

                results_data_MTBE = results_data_VOC_op

                if drills_list == []:
                    results_data_MTBE['Analyte'] = results_data_MTBE['Analyte'].apply(normalize_drill_name)
                    drills_list = results_data_MTBE["Analyte"].dropna().tolist()[1:]
                else:
                    results_data_MTBE['Analyte'] = results_data_MTBE['Analyte'].apply(normalize_drill_name)
                    mtbe_drills_list = results_data_MTBE["Analyte"].dropna().tolist()[1:]
                    for mtbe_item in mtbe_drills_list:
                        if not any(mtbe_item in drill_item for drill_item in drills_list):
                            drills_list.append(mtbe_item)

                results_data_VOC = results_data_VOC_op.T
                results_data_VOC.iloc[:, [0, 1]] = results_data_VOC.iloc[:, [1, 0]].values

        else:
            display(Javascript('alert("⚠️ MTBE BTEX is not in the file results you uploaded ⚠️")'))

if Add_exceptations_VOCs == True:
    df_res_all_cleaned = df_res_all.reset_index(drop=True)

    # Initialize a list to store DataFrames for each occurrence
    results_data_VOCs_dict = {}
    occurrence_count = 0

    # Dynamically find the row containing 'בדיקה' and ensure 'גבול גילוי' exists four columns to the right
    start_row = None
    col_index_בדיקה = None

    for row_index, row in df_res_all_cleaned.iterrows():
        if 'ב ד י ק ה' in row.values:
            col_name_בדיקה = row[row == 'ב ד י ק ה'].index[0]  # Get column name of 'בדיקה'
            col_index_בדיקה = df_res_all_cleaned.columns.get_loc(col_name_בדיקה)  # Get column index of 'בדיקה'
            col_index_גילוי = col_index_בדיקה + 4  # Four columns to the right

            # Validate if 'גבול גילוי' exists at the correct position (strip whitespace)
            if col_index_גילוי < len(df_res_all_cleaned.columns) and row.iloc[col_index_גילוי].strip() == 'גבול גילוי':
                start_row = row_index




                בדיקה_row = start_row

                # Find the end row by stopping at the first NaN in the 'בדיקה' column
                start_row += 3
                end_row = start_row

                for i in range(start_row, len(df_res_all_cleaned)):
                    value = df_res_all_cleaned.iloc[i, col_index_בדיקה + 1]
                    if pd.isna(value):  # Stop at the first empty cell
                        break
                    end_row += 1

                # Find the end column by stopping at the first NaN in the relevant row
                start_col = col_index_גילוי + 2
                end_col = start_col

                for i in range(start_col, len(df_res_all_cleaned.columns)):
                    value = df_res_all_cleaned.iloc[בדיקה_row + 1, i]
                    if pd.isna(value):  # Stop at the first empty cell
                        break
                    end_col += 1

                # Extract the relevant rows and columns dynamically using `.iloc`
                results_data_VOC = df_res_all_cleaned.iloc[
                    list(range(בדיקה_row + 1, end_row)),
                    list(range(col_index_בדיקה +1, end_col))
                ]

                results_data_VOC = results_data_VOC.reset_index(drop=True)
                results_data_VOC.iloc[0] = results_data_VOC.iloc[0].apply(normalize_drill_name)

                # Define the function to replace "ND" based on the value in column 4
                def replace_ND(cell, row_index):
                    if isinstance(cell, str) and "ND" in cell:
                        return f"<{results_data_VOC.iloc[row_index, 3]}"  # Replace with the value in column 3 of the same row
                    return cell

                # Iterate over each column (excluding column 3 itself) and apply the replacement
                for col in results_data_VOC.columns:  # Iterate over all columns
                    if results_data_VOC.columns.get_loc(col) != 3:  # Skip column 3
                        results_data_VOC[col] = results_data_VOC.apply(
                            lambda row: replace_ND(row[col], row.name), axis=1
                        )

                results_data_VOC_op = results_data_VOC.T
                results_data_VOC_op = results_data_VOC_op.reset_index(drop=True)

                # Drop the 1-4 rows
                results_data_VOC_op = results_data_VOC_op.drop(index=range(2, 5))
                results_data_VOC_op = results_data_VOC_op.drop(columns=results_data_VOC_op.columns[1])
                results_data_VOC_op = results_data_VOC_op.reset_index(drop=True)
                #results_data_VOC_op[0] = results_data_VOC_op[0].apply(normalize_drill_name)

                if occurrence_count > 0:
                  results_data_VOC_op = results_data_VOC_op.drop(index=range(0,2))

                # Save the DataFrame with a unique name in the dictionary
                results_data_VOCs_dict[f"results_data_VOC_{occurrence_count}"] = results_data_VOC_op

                # Increment the counter for the next DataFrame
                occurrence_count += 1
    if results_data_VOCs_dict:  # Check if there are any DataFrames to concatenate
    # Concatenate all DataFrames in the dictionary as new rows
      results_data_VOC_op = pd.concat(results_data_VOCs_dict.values(), ignore_index=True)

    if drills_list == []:
      #results_data_VOC_op[0] = results_data_VOC_op[0].apply(normalize_drill_name)
      drills_list = results_data_VOC_op[0].dropna().tolist()
      #drills_list = normalize_drills_list(drills_list)

      #drills_list = drills_list[1:]
    if not results_data_VOC_op.empty:
      #results_data_VOC_op[0] = results_data_VOC_op[0].apply(normalize_drill_name)
      VOC_drills_list = results_data_VOC_op[0].dropna().tolist()
      #VOC_drills_list = normalize_drills_list(VOC_drills_list)


      for VOC_item in VOC_drills_list:
          # Check if voc_item is in any item of drills_list
          if not any(VOC_item in drill_item for drill_item in drills_list):
              drills_list.append(VOC_item)  # Add to drills_list if no match is found

      results_data_VOC = results_data_VOC_op.T
      results_data_VOC.iloc[:, [0, 1]] = results_data_VOC.iloc[:, [1, 0]].values
      results_data_VOC_op = results_data_VOC_op.drop(index=0)

    # Raise an error if 'בדיקה' with 'גבול גילוי' was not found
    if start_row is None:
        display(Javascript('alert("⚠️ VOC is not in the file results you uploaded ⚠️")'))

# Process each item in drills_list
df_template['full name'] = None

# Updated extraction logic for name, depth, and type
def extract_name_and_depth(item):
    if not isinstance(item, str):
        return item, None, None

    # Detect flags
    is_dup = bool(re.search(r'\b(?:dup|duplicate|DUP|D)\b', item, re.IGNORECASE))
    is_split = bool(re.search(r'\b(?:split|SP)\b', item, re.IGNORECASE))

    # Clean flags from item for name extraction
    clean_item = re.sub(r'\b(?:dup|duplicate|DUP|D|split|SP)\b', '', item, flags=re.IGNORECASE).strip()

    # Match name and depth
    match_paren = re.search(r'(.+?)\s*\((\d+\.\d+)\)', clean_item)
    if match_paren:
        name = match_paren.group(1).strip()
        depth = match_paren.group(2)
    else:
        match_dash = re.search(r'^(.+?-\d+\.\d+)\s*-\s*(\d+\.\d+)', clean_item)
        if match_dash:
            name = match_dash.group(1).strip()
            depth = match_dash.group(2)
        else:
            match = re.findall(r'(\d+\.\d+)', clean_item)
            if match:
                depth = match[-1]
                name = clean_item[:clean_item.rfind(depth)].strip(" -")
            else:
                name, depth = clean_item.strip(), None

    return name, depth, ('**' if is_split else '*') if is_dup or is_split else ''

# Build a temp list to group regular, DUP, and split rows
temp_rows = {}

for item in drills_list:
    name, depth, suffix = extract_name_and_depth(item)
    if depth is None:
        continue
    key = (name, depth)

    # Initialize group
    if key not in temp_rows:
        temp_rows[key] = []

    # Compose adjusted depth
    depth_display = f"{depth}{suffix}"
    temp_rows[key].append({
        'name': name,
        'depth': depth_display,
        'full name': item
    })

# Flatten into df_template in the order: original, DUP, split
for key in sorted(temp_rows.keys()):
    # Ensure order: normal -> * -> **
    ordered = sorted(temp_rows[key], key=lambda x: len(x['depth']))
    for row_data in ordered:
        new_row = {**row_data}
        df_template = pd.concat([df_template, pd.DataFrame([new_row])], ignore_index=True)

# Ensure columns match
df_template = df_template.reindex(columns=columns, fill_value=None)


# Define the starting row for sorting
start_row = 4

# Extract the header rows (rows before start_row) and the rows to be sorted
header_rows = df_template.iloc[:start_row]
rows_to_sort = df_template.iloc[start_row:]

# Extract base name (e.g., ק) and numeric part as float (e.g., 1.1 from ק-1.1)
rows_to_sort['name_prefix'] = rows_to_sort['name'].str.extract(r'([^-\d]+)', expand=False)
rows_to_sort['name_number'] = rows_to_sort['name'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float)

# Sort by prefix, then by numeric name, then by depth
sorted_rows = rows_to_sort.sort_values(
    by=['name_prefix', 'name_number', 'depth'],
    key=lambda col: pd.to_numeric(col, errors='coerce') if col.name in ['name_number', 'depth'] else col
)

# Drop the temporary columns used for sorting
sorted_rows = sorted_rows.drop(columns=['name_prefix', 'name_number'])

# Concatenate the header rows with the sorted rows
df_sorted = pd.concat([header_rows, sorted_rows], ignore_index=True)

if 'TPH' in testsׁ_basic and not results_data_TPH.empty:
  df_sorted['TPH DRO'] = None
  df_sorted['TPH ORO'] = None
  df_sorted['Total TP'] = None

  # Ensure column names are consistent and merge based on "full name" and "Analyte"
  merged_df = df_sorted.merge(
      results_data_TPH.rename(columns={"Analyte": "full name"}),
      on="full name",
      how="left"
  )

  # Update the relevant columns in df_sorted with data from results_data_TPH
  df_sorted['TPH DRO'] = merged_df['DRO']
  df_sorted['TPH ORO'] = merged_df['ORO']
  df_sorted['Total TP'] = merged_df['Total TPH']

  # Renaming the column 'Total TP' to 'Total TPH'
  df_sorted.rename(columns={'Total TP': 'Total TPH'}, inplace=True)

  #Adding CAS number
  df_sorted['TPH DRO'][1] = "C10-C40"
  df_sorted['TPH ORO'][1] = "C10-C40"
  df_sorted['Total TPH'][1] = "C10-C40"

if 'MTBE' in testsׁ_basic and not results_data_MTBE.empty:
  df_sorted['MTBE_1'] = None
  df_sorted['Benzene_1'] = None
  df_sorted['Toluene_1'] = None
  df_sorted['Ethylbenzene_1'] = None
  df_sorted['Xylene_1'] = None

  # Ensure column names are consistent and merge based on "full name" and "Analyte"
  merged_df = df_sorted.merge(
      results_data_MTBE.rename(columns={"Analyte": "full name"}),
      on="full name",
      how="left"
  )

  # Update the relevant columns in df_sorted with data from results_data_TPH
  df_sorted['MTBE_1'] = merged_df['MTBE']
  df_sorted['Benzene_1'] = merged_df['Benzene']
  df_sorted['Toluene_1'] = merged_df['Toluene']
  df_sorted['Ethylbenzene_1'] = merged_df['Ethylbenzene']
  df_sorted['Xylene_1'] = merged_df['Xylene']

  # Renaming the columns
  df_sorted.rename(columns={'MTBE_1': 'MTBE'}, inplace=True)
  df_sorted.rename(columns={'Benzene_1': 'Benzene'}, inplace=True)
  df_sorted.rename(columns={'Toluene_1': 'Toluene'}, inplace=True)
  df_sorted.rename(columns={'Ethylbenzene_1': 'Ethylbenzene'}, inplace=True)
  df_sorted.rename(columns={'Xylene_1': 'Xylene'}, inplace=True)

  #Adding CAS number
  df_sorted['MTBE'][1] = '1634-04-4'
  df_sorted['Benzene'][1] = '71-43-2'
  df_sorted['Toluene'][1] = '108-88-3'
  df_sorted['Ethylbenzene'][1] = '100-41-4'
  df_sorted['Xylene'][1] = '95-47-6'


if Add_exceptations_VOCs == True and not results_data_VOC_op.empty:
    # Ensure CAS numbers in both dataframes are properly formatted
    results_data_VOC_op.iloc[0, :] = results_data_VOC_op.iloc[0, :].astype(str).str.strip()
    threshold_data['CAS No.'] = threshold_data['CAS No.'].astype(str).str.strip()

    # Initialize a new row with NaN values
    new_row = [None] * results_data_VOC_op.shape[1]

    # Iterate through each CAS number in the first row of results_data_VOC_op
    for col_index, cas_number in enumerate(results_data_VOC_op.iloc[0, :]):
        if pd.isna(cas_number) or cas_number == 'nan':
            continue

        # Find the matching row in threshold_data
        matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]
        if not matching_row.empty:
            # Get the value from the "VSL" column
            vsl_value = matching_row.iloc[0]['VSL']
            new_row[col_index] = vsl_value

    # Append the new row to results_data_VOC_op
    results_data_VOC_op.loc[len(results_data_VOC_op) + 1] = new_row

    ### Adding the values
    # Iterate through each column in results_data_VOC_op
    for col_index in range(results_data_VOC_op.shape[1]):
        cas_number = results_data_VOC_op.iloc[0, col_index]  # Get the CAS number from row 0
        if pd.isna(cas_number) or cas_number == 'nan':
            continue

        # Check if the CAS number is already in df_sorted
        if cas_number in df_sorted.iloc[1, :].values:
            continue  # Skip if the CAS number is already present

        # Get the column values excluding the first row (CAS number)
        column_values = results_data_VOC_op.iloc[1:, col_index]

        # Skip columns where all values are NaN
        if column_values.isna().all():
            continue

        # Filter column values to keep only numeric ones
        numeric_values = pd.to_numeric(column_values, errors='coerce')

        # Get the VSL value (assuming it's in the last row of the column)
        vsl_value = results_data_VOC_op.iloc[-1, col_index]
        if vsl_value is not None and not pd.isna(vsl_value):  # Check for None or NaN
            try:
                vsl_value = float(vsl_value)  # Ensure VSL value is a float
            except ValueError:
                continue  # Skip columns where VSL is not a valid number
        else:
            continue  # Skip if vsl_value is None or NaN

        # **Check for numeric values exceeding VSL**
        if (numeric_values > vsl_value).any():
            matching_row = results_data_VOC[results_data_VOC.iloc[:, 1] == cas_number]
            header_name = matching_row.iloc[0, 0] if not matching_row.empty else f"Column_{cas_number}"

            # Add new column to df_sorted
            df_sorted[header_name] = None
            df_sorted.at[1, header_name] = cas_number

            # Fill values where 'full name' matches
            for index, row in df_sorted.iterrows():
                if index <= 1:
                    continue
                name = row['full name']
                if pd.isna(name):
                    continue

                matching_row = results_data_VOC_op[
                    results_data_VOC_op[0].apply(lambda x: x in name if pd.notna(x) else False)
                ]
                if matching_row.empty:
                    continue

                value_to_fill = matching_row.iloc[0, col_index]
                df_sorted.at[index, header_name] = value_to_fill

        # **Check for "<..." values where the extracted number is greater than VSL**
        for cell_value in column_values:
            if isinstance(cell_value, str) and cell_value.startswith("<"):
                match = re.search(r'<\s*(\d+(\.\d+)?)', cell_value)
                if match:
                    try:
                        extracted_value = float(match.group(1))  # Extracted number from "<..."

                        if extracted_value > vsl_value:  # Compare with VSL
                            # Add column if condition is met
                            matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]
                            header_name = (
                                matching_row.iloc[0, -1]
                                if not matching_row.empty
                                else f"Column_{cas_number}_BDL"
                            )

                            # Ensure column exists in df_sorted
                            if header_name not in df_sorted.columns:
                                df_sorted[header_name] = None
                                df_sorted.at[1, header_name] = cas_number

                            # Fill values where 'full name' matches
                            for index, row in df_sorted.iterrows():
                                if index <= 1:
                                    continue
                                name = row['full name']
                                if pd.isna(name):
                                    continue

                                matching_row = results_data_VOC_op[
                                    results_data_VOC_op[0].apply(lambda x: x in name if pd.notna(x) else False)
                                ]
                                if matching_row.empty:
                                    continue

                                value_to_fill = matching_row.iloc[0, col_index]
                                df_sorted.at[index, header_name] = value_to_fill

                    except ValueError:
                        pass  # Skip invalid values

if Add_specifics_VOCs and not results_data_VOC_op.empty :
    Add_specifics_VOCs_list = [item.strip() for item in Add_specifics_VOCs.split(',')]
    for item in Add_specifics_VOCs_list:
        if item not in results_data_VOC[1].values:
            display(Javascript('alert("⚠️ one or more of the specific CAS numbers you insert is not in lab results ⚠️")'))
            print(f'{item} is not in lab results')
        else:
            # Find the column index of the item in results_data_VOC_op
            try:
                col_index = results_data_VOC_op.iloc[0, :].tolist().index(item)
            except ValueError:
                print(f'{item} not found in results_data_VOC_op header.')
                continue

            # Check if the CAS number (item) is already in df_sorted
            if item in df_sorted.iloc[1, :].values:
                continue  # Skip if the CAS number is already present

            # Find the row in results_data_VOC where the CAS number matches
            matching_row = results_data_VOC[
                results_data_VOC.iloc[:, 1] == item  # Match CAS number in column 1
            ]

            # If a match is found, get the header from column 0 in the matching row
            if not matching_row.empty:
                header_name = matching_row.iloc[0, 0]
            else:
                header_name = f"Column_{item}"  # Fallback to a generic name

            # Add a new column to df_sorted with the header_name
            df_sorted[header_name] = None
            df_sorted.at[1, header_name] = item

            # Add matching values to df_sorted for the rows where 'full name' matches
            for index, row in df_sorted.iterrows():
                if index <= 1:  # Skip the header and CAS rows
                    continue
                name = row['full name']
                if pd.isna(name):
                    continue

                # Find the matching row in results_data_VOC_op
                matching_row = results_data_VOC_op[
                    results_data_VOC_op[0].apply(lambda x: x in name if pd.notna(x) else False)
                ]
                if matching_row.empty:
                    continue

                # Extract the value from results_data_VOC_op for the current column
                value_to_fill = matching_row.iloc[0, col_index]
                df_sorted.at[index, header_name] = value_to_fill

df_sorted.iloc[0, 6:] = 'mg/kg'

# Iterate through each column in df_sorted starting from column 6 onward
for col_index in range(6, len(df_sorted.columns)):
    cas_number = df_sorted.iat[1, col_index]  # Get the CAS number from row 1

    if pd.isna(cas_number) or cas_number == '':  # Skip if CAS number is empty or NaN
        continue

    # Find the matching row in threshold_data based on CAS No.
    matching_row = threshold_data[threshold_data['CAS No.'] == cas_number]

    if not matching_row.empty:
        # Extract the VSL value from the matching row
        vsl_value = matching_row.iloc[0]['VSL']
        # Populate the cell in row 2 of df_sorted for the current column
        df_sorted.iat[2, col_index] = vsl_value

        # Extract the value from the column with the name Tier1_type
        if Tier1_type in matching_row.columns:
            tier1_value = matching_row.iloc[0][Tier1_type]
            # Populate the cell in row 3 of df_sorted for the current column
            df_sorted.iat[3, col_index] = tier1_value


df_sorted = df_sorted.drop(columns='full name')

# Add '*' to duplicate (name, depth) rows in df_sorted from row 5 onward
seen_pairs = {}
for idx in range(4, len(df_sorted)):
    name = df_sorted.at[idx, 'name']
    depth = str(df_sorted.at[idx, 'depth']).replace('*', '')  # strip existing stars
    key = (name, depth)

    if key in seen_pairs:
        # If already seen, mark this one with *
        df_sorted.at[idx, 'depth'] = f"{depth}*"
    else:
        seen_pairs[key] = idx  # store first occurrence

########סידור לטבלת אקסל להורדה########

# Fill empty values from row 5 (index 4) and column 6 onward with "-"
for row_idx in range(4, len(df_sorted)):
    for col_idx in range(5, len(df_sorted.columns)):
        val = df_sorted.iat[row_idx, col_idx]
        if pd.isna(val) or val == '':
            df_sorted.iat[row_idx, col_idx] = "-"

# Fill empty values in rows 3 and 4 (index 2 and 3) and column 6 onward with "--"
for row_idx in [2, 3]:
    for col_idx in range(5, len(df_sorted.columns)):
        val = df_sorted.iat[row_idx, col_idx]
        if pd.isna(val) or val == '':
            df_sorted.iat[row_idx, col_idx] = "--"

from openpyxl import Workbook
from openpyxl.styles import Font, Border, Side, Alignment, PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
import pandas as pd

# Create an Excel workbook and add a worksheet
wb = Workbook()
ws = wb.active

# Set the sheet to right-to-left
ws.sheet_view.rightToLeft = True

thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

alignment_center_wrap = Alignment(horizontal='center', vertical='center', wrap_text=True)

# Define colors for conditional formatting
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
custom_fill = PatternFill(start_color="F7C7AC", end_color="F7C7AC", fill_type="solid")
gray_fill = PatternFill(start_color="D3D3D3", end_color="D3D3D3", fill_type="solid")

# Step 1: Add the merged cells for the headers
ws.merge_cells(start_row=1, start_column=1, end_row=5, end_column=1)
cell = ws.cell(row=1, column=1)
cell.value = "שם קידוח"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

for i in range(2, 6):
  cell = ws.cell(row=i, column=1)
  cell.border = Border(
      left=Side(style='thin'),
      right=Side(style='thin'))

for i in range(2, 6):  # Iterate over columns (2 to 5 inclusive)
    for j in range(1, len(df_sorted)+2):  # Iterate over rows (1 to len(df_sorted) - 1)
        cell = ws.cell(row=j, column=i)
        cell.border = Border(bottom=Side(style='thin'))

ws.merge_cells(start_row=1, start_column=2, end_row=5, end_column=2)
cell = ws.cell(row=1, column=2)
cell.value = "תאריך"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=3)
cell.value = ""
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=3)
cell.value = "יחידות"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=4)
cell.value = "עומק"
cell.font = Font(name=Font_name_hebrew, size=Font_size_hebrew, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=4)
cell.value = "m"
cell.font = Font(name=Font_name_english, size=Font_size_english)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=1, column=5)
cell.value = "PID"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

cell = ws.cell(row=2, column=5)
cell.value = "ppm"
cell.font = Font(name=Font_name_english, size=Font_size_english)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=3, start_column=3, end_row=3, end_column=5)
cell = ws.cell(row=3, column=3)
cell.value = "CAS"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=4, start_column=3, end_row=4, end_column=5)
cell = ws.cell(row=4, column=3)
cell.value = "VSL"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

ws.merge_cells(start_row=5, start_column=3, end_row=5, end_column=5)
cell = ws.cell(row=5, column=3)
cell.value = "TIER 1"
cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
cell.alignment = alignment_center_wrap
cell.border = thin_border

# Helper function to detect if a string contains English letters
def contains_english(value):
    return any(char.isascii() and char.isalpha() for char in str(value))

# Step 2: Add the 'name' column to the first column in the Excel file starting from row 4 of df_sorted
for row_idx, value in enumerate(df_sorted["name"][4:], start=6):
    cell = ws.cell(row=row_idx, column=1)
    cell.value = value
    cell.font = Font(
        name=Font_name_english if contains_english(value) else Font_name_hebrew,
        size=Font_size_english if contains_english(value) else Font_size_hebrew,
    )
    cell.alignment = alignment_center_wrap
    cell.border = thin_border

# Merge cells in column 1 with the same value
current_start = 6
previous_value = None
for row_idx, value in enumerate(df_sorted["name"][4:], start=6):
    if value == previous_value:
        continue
    if previous_value is not None:
        ws.merge_cells(start_row=current_start, start_column=1, end_row=row_idx - 1, end_column=1)
    current_start = row_idx
    previous_value = value
# Merge the last group
if previous_value is not None:
    ws.merge_cells(start_row=current_start, start_column=1, end_row=len(df_sorted["name"][4:]) + 5, end_column=1)

# Step 3: Add the 'depth' column to column 4 in the Excel file starting from row 6
for row_idx, value in enumerate(df_sorted["depth"][4:], start=6):
    cell = ws.cell(row=row_idx, column=4)
    cell.value = value
    cell.font = Font(
        name=Font_name_english if contains_english(value) else Font_name_hebrew,
        size=Font_size_english if contains_english(value) else Font_size_hebrew,
    )
    cell.alignment = alignment_center_wrap
    cell.border = thin_border

# Step 4: Add data from column 6 onward starting from row 1 in Excel
for col_idx, col_name in enumerate(df_sorted.columns[5:], start=6):  # Iterate through columns 6 onward
    # Add column header
    header_cell = ws.cell(row=1, column=col_idx)
    header_cell.value = col_name
    header_cell.font = Font(name=Font_name_english, size=Font_size_english, bold=True)
    header_cell.alignment = alignment_center_wrap
    header_cell.border = thin_border

    # Add column data and round float values
    for row_idx, value in enumerate(df_sorted[col_name], start=2):  # Start from row 2 in Excel
        cell = ws.cell(row=row_idx, column=col_idx)
        try:
            value_numeric = round(float(value), 4)  # Round to 4 decimal places
            cell.value = value_numeric
        except (ValueError, TypeError):
            cell.value = value  # Non-numeric values remain unchanged

        cell.font = Font(
            name=Font_name_english if contains_english(value) else Font_name_hebrew,
            size=Font_size_english if contains_english(value) else Font_size_hebrew,
        )
        cell.alignment = alignment_center_wrap
        cell.border = thin_border

        # Conditional formatting for rows 6 and onward
        if row_idx >= 6:
            try:
                row_4_value = float(ws.cell(row=4, column=col_idx).value)
                row_5_value = float(ws.cell(row=5, column=col_idx).value)

                if isinstance(cell.value, (float, int)):
                    if cell.value > row_4_value:
                        cell.fill = yellow_fill
                        cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)
                    if cell.value > row_5_value:
                        cell.fill = custom_fill
                        cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)

                # **Gray Fill Logic for "<..." values (Handles both Integers & Floats)**
                elif isinstance(cell.value, str) and cell.value.startswith("<"):
                    match = re.search(r'<\s*(\d+(\.\d+)?)', cell.value)  # Extract numeric part

                    if match:
                        try:
                            numeric_value = float(match.group(1))  # Convert extracted value to float
                            threshold_value = ws.cell(row=4, column=col_idx).value  # Get row 4 value

                            # Ensure threshold value is numeric before comparison
                            if isinstance(threshold_value, (int, float)) and numeric_value > threshold_value:
                                cell.fill = gray_fill
                                cell.font = Font(bold=True, name=cell.font.name, size=cell.font.size)
                        except ValueError:
                            pass  # Skip if conversion fails

            except (ValueError, TypeError):
                pass  # Skip cells that cannot be compared numerically

# Merge columns 2 and 3 for each row from row 6 onward
for row_idx in range(6, len(df_sorted["name"]) + 6):
    ws.merge_cells(start_row=row_idx, start_column=2, end_row=row_idx, end_column=3)

# Define the medium border
medium_side = Side(style="medium")

# Define the range for the table
start_row = 1  # Start from the first row
end_row = len(df_sorted)+1  # Dynamically determine the last row based on the data
start_col = 1  # Start from the first column
end_col = ws.max_column  # Dynamically determine the last column based on the data

# Apply medium border to the outer edges of the table
for row in range(start_row, end_row + 1):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)

        # Apply borders to the outer edges only
        if row == start_row:  # Top edge
            cell.border = Border(top=medium_side, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
        if row == end_row:  # Bottom edge
            cell.border = Border(bottom=medium_side, left=cell.border.left, right=cell.border.right, top=cell.border.top)
        if col == start_col:  # Left edge
            cell.border = Border(left=medium_side, top=cell.border.top, bottom=cell.border.bottom, right=cell.border.right)
        if col == end_col:  # Right edge
            cell.border = Border(right=medium_side, top=cell.border.top, bottom=cell.border.bottom, left=cell.border.left)

# Define the range for the first 5 rows
start_row = 1
end_row = 5
start_col = 1
end_col = ws.max_column  # Dynamically determine the last column based on the data

# Apply thick border to the outer edges
for row in range(start_row, end_row + 1):
    for col in range(start_col, end_col + 1):
        cell = ws.cell(row=row, column=col)

        # Apply borders to the outer edges only
        if row == start_row:  # Top edge
            cell.border = Border(top=medium_side, left=cell.border.left, right=cell.border.right, bottom=cell.border.bottom)
        if row == end_row:  # Bottom edge
            cell.border = Border(bottom=medium_side, left=cell.border.left, right=cell.border.right, top=cell.border.top)
        if col == start_col:  # Left edge
            cell.border = Border(left=medium_side, top=cell.border.top, bottom=cell.border.bottom, right=cell.border.right)
        if col == end_col:  # Right edge
            cell.border = Border(right=medium_side, top=cell.border.top, bottom=cell.border.bottom, left=cell.border.left)

# Determine the last row and column of the table
last_row = ws.max_row
start_col = 1  # Assuming you want the legend to start from the first column

# Add legend for DUP and Split notation
legend_dup_split = ws.cell(row=len(df_sorted)+2, column=start_col)
legend_dup_split.value = "*Duplicate | **Split"
legend_dup_split.font = Font(bold=True, name=Font_name_english, size=Font_size_english)
legend_dup_split.alignment = alignment_center_wrap
legend_dup_split.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for - לא בוצעה אנליזה
legend_dup_split = ws.cell(row=len(df_sorted)+3, column=start_col)
legend_dup_split.value = " - לא בוצעה אנליזה"
legend_dup_split.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_dup_split.alignment = alignment_center_wrap
legend_dup_split.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for ' -- לא קיים ערך סף
legend_dup_split = ws.cell(row=len(df_sorted)+4, column=start_col)
legend_dup_split.value = " -- לא קיים ערך סף"
legend_dup_split.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_dup_split.alignment = alignment_center_wrap
legend_dup_split.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "חורג מערך VSL"
legend_vsl_cell = ws.cell(row=len(df_sorted)+5 , column=start_col)
legend_vsl_cell.value = "חורג מערך VSL"
legend_vsl_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_vsl_cell.alignment = alignment_center_wrap
legend_vsl_cell.fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
legend_vsl_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "חורג מערך TIER 1"
legend_tier1_cell = ws.cell(row=len(df_sorted)+6 , column=start_col)
legend_tier1_cell.value = "חורג מערך TIER 1"
legend_tier1_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_tier1_cell.alignment = alignment_center_wrap
legend_tier1_cell.fill = PatternFill(start_color="F7C7AC", end_color="F7C7AC", fill_type="solid")
legend_tier1_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Add legend for "ערך גילוי גדול מערך סף"
legend_gray_cell = ws.cell(row=len(df_sorted)+7, column=start_col)
legend_gray_cell.value = "ערך סף גילוי מעבדה גדול מערך סף VSL"
legend_gray_cell.font = Font(bold=True, name=Font_name_hebrew, size=Font_size_hebrew)
legend_gray_cell.alignment = alignment_center_wrap
legend_gray_cell.fill = gray_fill
legend_gray_cell.border = Border(
    left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin")
)

# Save the final Excel file with the provided name
file_path = f'{file_name}.xlsx'
wb.save(file_path)

# Download the Excel file
files.download(file_path)


<ipython-input-17-4094318418>:464: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_to_sort['name_prefix'] = rows_to_sort['name'].str.extract(r'([^-\d]+)', expand=False)
<ipython-input-17-4094318418>:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rows_to_sort['name_number'] = rows_to_sort['name'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float)
<ipython-input-17-4094318418>:500: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>